In [ ]:
from utils import *
import matplotlib.pyplot as plt

img_path = "../img/Outline/"
# Get the folder information
info = load_folder_information(img_path)
# Generate masks
origin = load_image(os.path.join(img_path, info["origin"]))

print(info)
plt.imshow(origin)

In [ ]:
mask_num = len(info["mask_prefix"])
masks = []
for i in range(mask_num):
    mask = np.zeros_like(origin)
    masked_column = origin.shape[1] // (mask_num + 1) * (i + 1)
    mask[:, masked_column:] = 1
    masks.append(mask)

# for line in range(masks[0].shape[0]):
#     for column in range(masks[0].shape[1]):
#         print(masks[0][line][column][0], end=" ")
#     print()
plt.imshow(masks[6]*255)

In [ ]:
# Image files
img_names = os.listdir(img_path)
# MSE
logits = []
for i in range(mask_num):
    mask = masks[i]
    # for line in range(mask.shape[0]):
    #     for column in range(mask.shape[1]):
    #         print(mask[line][column][0], end=" ")
    #     print()
    mask_prefix = info["mask_prefix"][i]
    imgs = [
        load_image(os.path.join(img_path, img_name))
        for img_name in img_names
        if mask_prefix in img_name
    ]
    mse = [image_MSE(origin, img, mask) for img in imgs]
    logits.append(np.mean(mse))
    # for img in imgs:
    #     plt.imshow(img)
    #     plt.show()

print(logits)

In [ ]:
hpd_path = "/nas/Public/data/HPDv2/test/"
save_path = "/home/lyy/Repos/FACE-image/data/"

import os

file_list = []

if not os.path.exists(save_path):
    os.mkdir(save_path)

for file in os.listdir(hpd_path):
    os.symlink(os.path.join(hpd_path, file), os.path.join(save_path, f"image-{int(file.split('.')[0])}." + file.split(".")[-1] if file.split(".")[-1] != "" else "jpg"))

## results analysis

In [ ]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from pygam import LinearGAM

SPLIT = 64
WIDTH = 512
MODEL_NUMBER = 9

hpd_path = "/nas/Public/data/HPDv2/test/"
generate_path = "/nas/Public/experiment_result/FACE-image/"
mapping_path = "/nas/Public/data/HPDv2/test.json"

In [ ]:
mapping = json.load(open(mapping_path))
files = os.listdir(generate_path)

groups = []
winners = []

for item in mapping:
    group_complete = True
    if len(item["image_path"]) != MODEL_NUMBER:
        # cannot determine which models are used
        continue
    for path in item["image_path"]:
        # iterate image numbers
        number = int(path.split('.')[0])
        single_complete = True
        for i in range(SPLIT-1):
            # check each mask re-generation
            if (f"image-{number}_mask-{i}_00001_.png") not in files:
                single_complete = False
                break
        if not single_complete:
            group_complete = False
            break
    if group_complete:
        groups.append(item["image_path"])
        idx = 0
        for i, rank in enumerate(item["rank"]):
            if rank == 0:
                idx = i
                break
        winners.append(idx)

In [ ]:
print(len(groups))
print(groups[0])
print(winners)

In [ ]:
for names in groups:
    for idx, name in enumerate(names):
        for mask in range(SPLIT-1):
            no = int(name.split('.')[0])
            os.symlink(os.path.join(generate_path, f"image-{no}_mask-{mask}_00001_.png"), os.path.join(f"../data/model_{idx}", f"image-{no}_mask-{mask}_00001_.png"))

In [ ]:
# missing_list = []
# for i in range(1, 3701):
#     for j in range(SPLIT-1):
#         if (f"image-{i}_mask-{j}_00001_.png") not in files:
#             if i not in missing_list:
#                 missing_list.append(i)

### logits calc

In [ ]:
# logits = {}

# for i in range(MODEL_NUMBER):
#     logits[i] = []

# for group in tqdm(groups):
#     for i, file in enumerate(group):
#         subject_img = cv2.imread(os.path.join(generate_path, f"image-{int(file.split('.')[0])}.jpg_00001_.png"), cv2.IMREAD_COLOR)[:,:,::-1]
#         idx = int(file.split('.')[0])
#         # plt.imshow(subject_img)
#         # plt.show()
#         # print(idx)
#         sequence = []
#         # each mask strip
#         for mask_idx in range(SPLIT - 1):
#             generated_img = cv2.imread(os.path.join(generate_path, f"image-{idx}_mask-{mask_idx}_00001_.png"), cv2.IMREAD_COLOR)[:,:,::-1]
#             # one strip
#             columns = WIDTH // SPLIT * (mask_idx + 1)
#             token = np.array(generated_img[:, columns:columns + WIDTH // SPLIT], dtype=np.int16)
#             ground_truth = np.array(subject_img[:, columns:columns + WIDTH // SPLIT], dtype=np.int16)

#             mse = ((token - ground_truth) ** 2).mean()
#             sequence.append(mse)
#         logits[i].append(sequence)

# with open("./logits.json", "w") as f:
#     json.dump(logits, f)

In [ ]:
with open("./logits.json", "r") as f:
    logits = json.load(f)

### ranking clac

In [ ]:
rank_lists = {}
rank_avgs = {}
rank_var = {}
for i in range(MODEL_NUMBER):
    rank_lists[i] = []
for item in mapping:
    if len(item["rank"]) != MODEL_NUMBER:
        # cannot determine which models are used
        continue
    for i, rank in enumerate(item["rank"]):
        rank_lists[i].append(rank)

for i in range(MODEL_NUMBER):
    rank_avgs[i] = np.mean(rank_lists[i])
    rank_var[i] = np.var(rank_lists[i])

print(rank_avgs)
print(rank_var)

### plot

In [ ]:
logits_sum = []
spectra = []
for idx, winner in enumerate(winners):
    logits_sum.append(logits[str(winner)][idx])
    spectra.append(np.abs(np.fft.fft(logits[str(winner)][idx])))

logits_winner_avg = np.mean(logits_sum, axis=0)
spectrum_winner_avg = np.mean(spectra, axis=0)
# print(logits_winner_avg)
# print(spectrum_winner_avg)

In [ ]:
avg_logits = {}
fig_size = (15, 10)

plt.figure(0, fig_size)
plt.plot(np.linspace(0, len(logits_winner_avg), len(logits_winner_avg)), logits_winner_avg, label="winner avg", color="grey")
plt.legend()

spectrum_winner_avg = spectrum_winner_avg[1:]

plt.figure(1, fig_size)
plt.plot(np.linspace(0, 0.5, len(spectrum_winner_avg)), spectrum_winner_avg, label="winner avg", color="grey")
plt.legend()

X = np.linspace(0, 0.5, len(spectrum_winner_avg))
gam = LinearGAM().fit(X, spectrum_winner_avg)
XX = gam.generate_X_grid(term=0, n=len(spectra))
pdep, confi = gam.partial_dependence(term=0, X=XX, width=0.95)

# XX =       XX[10:]
# pdep =   pdep[10:]
# confi = confi[10:]

plt.figure(2, fig_size)
plt.plot(
    XX[:, 0],
    np.abs(pdep),
    label="winner avg",
    color="grey"
)
# plt.fill_between(
#     XX[:, 0],
#     np.where(
#         confi[:, 0] * confi[:, 1] > 0,
#         np.minimum(np.abs(confi[:, 0]), np.abs(confi[:, 1])),
#         0
#     ),
#     np.maximum(np.abs(confi[:, 0]), np.abs(confi[:, 1])),
#     alpha=0.25
# )
plt.legend()


for i, logit_list in logits.items():
    # print(len(logit_list[0]))
    # break
    avg_logits[i] = np.mean(logit_list, axis=0)

    plt.figure(0, fig_size)
    plt.title("Raw MSEs")
    plt.plot(np.linspace(0, len(avg_logits[i]), len(avg_logits[i])), avg_logits[i], label=f"avg: {rank_avgs[int(i)]:.3}, var: {rank_var[int(i)]:.3}")
    plt.legend()


    spectra = []
    for line in logit_list:
        spectrum = np.abs(np.fft.fft(line))
        spectra.append(spectrum)
    mean_spectrum = np.mean(spectra, axis=0)

    mean_spectrum = mean_spectrum[1:]

    plt.figure(1, fig_size)
    plt.title("Raw Spectra")
    plt.plot(np.linspace(0, 0.5, len(mean_spectrum)), mean_spectrum, label=f"avg: {rank_avgs[int(i)]:.3}, var: {rank_var[int(i)]:.3}")
    plt.legend()

    X = np.linspace(0, 0.5, len(mean_spectrum))
    gam = LinearGAM().fit(X, mean_spectrum)
    XX = gam.generate_X_grid(term=0, n=len(spectra))
    pdep, confi = gam.partial_dependence(term=0, X=XX, width=0.95)

    # XX =       XX[10:]
    # pdep =   pdep[10:]
    # confi = confi[10:]

    plt.figure(2, fig_size)
    plt.title("GAM Spectra")
    plt.plot(
        XX[:, 0],
        np.abs(pdep),
        label=f"avg: {rank_avgs[int(i)]:.3}, var: {rank_var[int(i)]:.3}",
    )
    # plt.fill_between(
    #     XX[:, 0],
    #     np.where(
    #         confi[:, 0] * confi[:, 1] > 0,
    #         np.minimum(np.abs(confi[:, 0]), np.abs(confi[:, 1])),
    #         0
    #     ),
    #     np.maximum(np.abs(confi[:, 0]), np.abs(confi[:, 1])),
    #     alpha=0.25
    # )
    plt.legend()
plt.show()

### score

In [ ]:
from scipy.stats import entropy

def spectral_overlap(spectrum1, spectrum2):
    spectra = np.concatenate([spectrum1[None], spectrum2[None]], axis=0)
    return spectra.min(axis=0).sum() / spectra.max(axis=0).sum()

def pearson_correlation(spectrum1, spectrum2):
    return np.corrcoef(spectrum1, spectrum2)[0, 1]

def earth_mover_distance(spectrum1, spectrum2):
    p1 = [spectrum1[0]]
    p2 = [spectrum2[0]]
    for value in spectrum1[1:]:
        p1.append(p1[-1] + value)
    for value in spectrum2[1:]:
        p2.append(p2[-1] + value)
    p1 = np.array(p1)
    p2 = np.array(p2)
    p1 /= p1[-1]
    p2 /= p2[-1]
    return np.abs(p1 - p2).sum() / spectrum1.shape[0]

def kl_divergence(spectrum1, spectrum2):
    kl = entropy(abs(spectrum1), abs(spectrum2))
    return kl

def score(spectrum1, spectrum2):
    so = spectral_overlap(spectrum1, spectrum2)
    corr = pearson_correlation(spectrum1, spectrum2)
    emd = earth_mover_distance(spectrum1, spectrum2)
    kl = kl_divergence(spectrum1, spectrum2)
    return so, corr, emd, kl

def area_under_curve(spectrum):
    return np.abs(spectrum).sum()

In [ ]:
mean_so_list = []
mean_corr_list = []
mean_emd_list = []
mean_kl_list = []

so_lists = []
corr_lists = []
emd_lists = []
kl_lists = []

for i, logit_list in logits.items():
    # print(len(logit_list[0]))
    # break
    avg_logits[i] = np.mean(logit_list, axis=0)

    so_list = []
    corr_list = []
    emd_list = []
    kl_list = []

    for line in logit_list:
        spectrum = np.abs(np.fft.fft(line))
        so, corr, emd, kl = score(spectrum_winner_avg, spectrum[1:])
        so_list.append(so)
        corr_list.append(corr)
        emd_list.append(emd)
        kl_list.append(kl)

    so_lists.append(so_list)
    corr_lists.append(corr_list)
    emd_lists.append(emd_list)
    kl_lists.append(kl_list)

    # stepwise mean scores
    mean_so = np.mean(so_list)
    mean_corr = np.mean(corr_list)
    mean_emd = np.mean(emd_list)
    mean_kl = np.mean(kl_list)

    mean_so_list.append(mean_so)
    mean_corr_list.append(mean_corr)
    mean_emd_list.append(mean_emd)
    mean_kl_list.append(mean_kl)

    print(f"SO: {mean_so:.5}\tCORR: {mean_corr:.5}\tEMD: {mean_emd:.5}\tKL: {mean_kl:.5}")

# stepwise rankings mean
so_ranks_idx = np.argsort(-np.array(so_lists), axis=0)
corr_ranks_idx = np.argsort(-np.array(corr_lists), axis=0)
emd_ranks_idx = np.argsort(np.array(emd_lists), axis=0)
kl_ranks_idx = np.argsort(np.array(kl_lists), axis=0)
so_ranks = np.zeros(so_ranks_idx.shape)
corr_ranks = np.zeros(corr_ranks_idx.shape)
emd_ranks = np.zeros(emd_ranks_idx.shape)
kl_ranks = np.zeros(kl_ranks_idx.shape)
for j in range(so_ranks_idx.shape[1]):
    for i in range(so_ranks_idx.shape[0]):
        so_ranks[so_ranks_idx[i][j]][j] = i
        corr_ranks[corr_ranks_idx[i][j]][j] = i
        emd_ranks[emd_ranks_idx[i][j]][j] = i
        kl_ranks[kl_ranks_idx[i][j]][j] = i

print('='*10 + " stepwise rankings " + '='*10)

for i in range(so_ranks.shape[0]):
    print(f"AVG: {(np.mean(so_ranks[i])+np.mean(corr_ranks[i])+np.mean(emd_ranks[i])+np.mean(kl_ranks[i]))/4:.3}\tSO: {np.mean(so_ranks[i]):.3}\tCORR: {np.mean(corr_ranks[i]):.3}\tEMD: {np.mean(emd_ranks[i]):.3}\tKL: {np.mean(kl_ranks[i]):.3}")

# mean scores ranking
so_ranks_idx = np.argsort(-np.array(mean_so_list))
corr_ranks_idx = np.argsort(-np.array(mean_corr_list))
emd_ranks_idx = np.argsort(mean_emd_list)
kl_ranks_idx = np.argsort(mean_kl_list)
so_ranks = np.zeros((len(so_ranks_idx)))
corr_ranks = np.zeros((len(corr_ranks_idx)))
emd_ranks = np.zeros((len(emd_ranks_idx)))
kl_ranks = np.zeros((len(kl_ranks_idx)))
for i in range(len(so_ranks_idx)):
    so_ranks[so_ranks_idx[i]] = i
    corr_ranks[corr_ranks_idx[i]] = i
    emd_ranks[emd_ranks_idx[i]] = i
    kl_ranks[kl_ranks_idx[i]] = i

print('='*10 + " rankings " + '='*10)

for i in range(len(so_ranks)):
    print(f"AVG: {(so_ranks[i]+corr_ranks[i]+emd_ranks[i]+kl_ranks[i])/4}\tSO: {so_ranks[i]}\tCORR: {corr_ranks[i]}\tEMD: {emd_ranks[i]}\tKL: {kl_ranks[i]}")

### inception score

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data
from torchvision.models.inception import inception_v3
import numpy as np
from tqdm import tqdm
from PIL import Image
import os
from scipy.stats import entropy

# we should use same mean and std for inception v3 model in training and testing process
# reference web page: https://pytorch.org/hub/pytorch_vision_inception_v3/

def imread(filename):
    """
    Loads an image file into a (height, width, 3) uint8 ndarray.
    """
    return np.asarray(Image.open(filename), dtype=np.uint8)[..., :3]

def inception_score(batch_size=50, resize=False, splits=10):
    # Set up dtype
    device = torch.device("cuda:1")  # you can change the index of cuda
    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).to(device)
    inception_model.eval()
    up = nn.Upsample(size=(299, 299), mode='nearest', align_corners=False).to(device)
    
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x, dim=1).data.cpu().numpy()

    # Get predictions using pre-trained inception_v3 model
    print('Computing predictions using inception v3 model')
    

    files = readDir()
    N = len(files)
    preds = np.zeros((N, 1000))
    if batch_size > N:
        print(('Warning: batch size is bigger than the data size. '
                 'Setting batch size to data size'))

    for i in tqdm(range(0, N, batch_size)):
        start = i
        end = i + batch_size
        images = np.array([imread(str(f)).astype(np.float32)
                           for f in files[start:end]])

        # Reshape to (n_images, 3, height, width)
        images = images.transpose((0, 3, 1, 2))
        images /= 255

        batch = torch.from_numpy(images).type(torch.FloatTensor)
        batch = batch.to(device)
        y = get_pred(batch)
        # print(y.shape)
        preds[i :i  + batch_size] = get_pred(batch)
        

    assert batch_size > 0
    assert N > batch_size

    # Now compute the mean KL Divergence
    print('Computing KL Divergence')
    split_scores = []
    for k in range(splits):
        part = preds[k * (N // splits): (k + 1) * (N // splits), :] # split the whole data into several parts
        py = np.mean(part, axis=0)  # marginal probability
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]  # conditional probability
            scores.append(entropy(pyx, py))  # compute divergence
        split_scores.append(np.exp(scores))

    return np.max(split_scores), np.mean(split_scores)

def readDir():
    dirPath = "/home/lyy/Repos/FACE-image/data/model_8"
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        for f in fileList:
            f = dirPath+'/'+f
            if os.path.isdir(f):
                subFiles = readDir(f)
                allFiles = subFiles + allFiles
            else:
                allFiles.append(f)
        return allFiles
    else:
        return 'Error,not a dir'

MAX, IS= inception_score(splits=10)
print('MAX IS is %.4f' % MAX)
print('The IS is %.4f' % IS)
